In [6]:
#Step 1 : Select kernet, Pyhton 3.8 which ever that is used in the environent

In [7]:
import os
import json
import pandas as pd
import traceback

In [8]:
from langchain.chat_models import ChatOpenAI

In [9]:
#from dotenv import load_dotenv
#load_dotenv()
#KEY=os.getenv("OPENAI_API_KEY")
KEY="sk-jey3RfzWi3WspUjs74kcT3B1bkFJsrJIcdRXSAABsy8rFFpj"

In [10]:
llm=ChatOpenAI(openai_api_key=KEY,model="gpt-3.5-turbo",temperature=0.5)